In [ ]:
import pandas as pd 
import pymysql 
import os 
from dotenv import load_dotenv

In [ ]:
# 환경 변수 설정 
load_dotenv()

In [ ]:
# mysql 서버에 접속 
_db = pymysql.connect(
    host = os.getenv('host'), 
    port = int(os.getenv('port')), 
    user = os.getenv('user'), 
    password = os.getenv('pwd'), 
    db = os.getenv('db_name')
)

In [ ]:
# 커서 생성 
cursor = _db.cursor(pymysql.cursors.DictCursor)

In [ ]:
select_sql = """
    SELECT 
    *
    FROM 
    `sales records`
    WHERE 
    `Sales Channel` = 'Online'
"""
cursor.execute(select_sql)

In [ ]:
cursor.fetchall()

In [ ]:
select_sql2 = """
    SELECT 
    * 
    FROM 
    `sales records`
    WHERE `Sales Channel` = %s
"""
values = ['Online', 'Offline']

cursor.execute(select_sql2, values)

In [ ]:
def select_query(
        _sql, 
        *_val
):
    # _sql : query문이 입력
    # _val : query에서 사용될 변수들의 모음(tuple)
    # 커서에 query문을 보낸다. 
    cursor.execute(_sql, _val)
    # query에 대한 응답을 가져온다. 
    cursor_data = cursor.fetchall()
    # cursor_data를 데이터프레임으로 변환 
    df = pd.DataFrame(cursor_data)
    # 데이터프레임을 되돌려준다. 
    return df

In [ ]:
query1 = """
    SELECT 
    * 
    FROM 
    `sales records` 
    WHERE `Country` in (%s, %s)
"""
select_query(query1, 'South Korea', 'North Korea')

In [ ]:
# emp table 에서 특정 조건에 맞는 데이터를 출력 
# SAL 필드에서 1500이상인 사원 정보를 출력
sql_query2 = """
    SELECT 
    * 
    FROM 
    `emp`
    WHERE `SAL` >= 1500
"""
select_query(sql_query2)


In [ ]:
sql_query = """
    SELECT 
    * 
    FROM 
    `emp`
"""
df = select_query(sql_query)
df

In [ ]:
# df에서 SAL 컬럼의 데이터가 1500이상인 데이터를 출력하시오
flag = df['SAL'] >= 1500
df.loc[flag]

In [ ]:
# sql 
# SAL 이 1500이상이고(and) 3000이하인 사원의 이름을 출력 
select_query3 = """
    SELECT 
    `ENAME`
    FROM 
    `emp`
    WHERE 
    `SAL` >= 1500 AND `SAL` <= 3000
"""
select_query(select_query3)

In [ ]:
sql_query3 = """
    SELECT
    `ENAME`
    FROM 
    `emp`
    WHERE `SAL` BETWEEN 1500 AND 3000
"""
select_query(sql_query3)

- pandas에서 데이터프레임 필터 
- df[조건식]
- df[컬럼명]
- loc[]
    - 인덱스의 조건식, 컬럼의 조건식
- iloc[]
    - 인덱스의 위치, 컬럼의 위치

In [ ]:
flag = df['SAL'] >= 1500
flag2 = df['SAL'] <= 3000

df[flag & flag2][['ENAME']]
df.loc[flag & flag2, ['ENAME']]

In [ ]:
# sql
# 사원의 이름이 S 시작하는 사원의 정보를 확인
sql_query4 = """
    SELECT 
    * 
    FROM 
    `emp`
    WHERE `ENAME` LIKE 'S%%'
"""
select_query(sql_query4)

In [ ]:
# pandas 
# 조건식 : ENAME 컬럼의 데이터가 S로 시작하는가?
# 문자형 내장함수 : startswith()
# 스리즈 형태 데이터에서 문자형 내장함수 사용 : series.str.함수명()
flag = df['ENAME'].str.startswith('S')

df.loc[flag]

In [ ]:
# ENAME에 데이터를 하나씩 뽑아서 앞의 단어가 S 인가?
word = 'S'

# map() 함수 
# series.map( 함수(일반함수, lambda함수) )
flag = df['ENAME'].map(
    # lambda x 의 x는 ENAME 값들이 하나씩 대입
    # if문을 한줄로 표현 : "참일경우 값" if 조건식 else "거짓일경우 값값"
    lambda x : True if x[ : len(word)] == word else False
)
df.loc[flag]

In [ ]:
word = 'S'
flag = []
for i in df['ENAME']:
    # i : ENAME의 값들이 하나씩 대입 
    flag.append(i[ : len(word)] == word)
df.loc[flag]

In [ ]:
flag = df['ENAME'].str.contains('S')
df.loc[flag]

In [ ]:
word = 'S'
flag = df['ENAME'].map(
    lambda x : word in x 
)
df.loc[flag]

In [ ]:
# join 
# 두개의 테이블을 결합하는 방식 중 하나 
# 특정한 조건에 맞춰서 테이블의 열을 결합하는 방식 
# pandas에서 merge()와 같은 기능
# emp table과 dept table을 결합 
sql_query5 = """
    SELECT 
    *
    FROM 
    `dept`
"""
dept = select_query(sql_query5)
dept

In [ ]:
df

In [ ]:
# emp, dept table join 결합 
# left join
join_query = """
    SELECT 
    a.`ENAME`, a.`JOB`, a.`SAL`, b.`DNAME`, b.`LOC`
    FROM 
    `emp` a
    LEFT JOIN 
    `dept` b
    ON a.`DEPTNO` = b.`DEPTNO`
"""
select_query(join_query)

In [ ]:
# pandas 
# pandas에서 merge(데이터프레임, 데이터프레임, on = 조건, how = 방식)
pd.merge(df, dept, on='DEPTNO', how = 'left')\
    [ ['ENAME', 'JOB', "SAL", 'DNAME', 'LOC'] ]
# Dataframe에서 merge(데이터프레임, on = 조건, how = 방식)
df.merge(dept, on='DEPTNO', how='left')\
    [ ['ENAME', 'JOB', "SAL", 'DNAME', 'LOC'] ]

In [68]:
# 외부의 데이터베이스에 접속 
_db2 = pymysql.connect(
    host = '172.30.1.56', 
    port = 3306, 
    user = 'ubion', 
    password = '1234', 
    db = 'ubion'
)

In [69]:
# 커서 생성
cursor2 = _db2.cursor(pymysql.cursors.DictCursor)

In [70]:
sql_query = """
    SELECT 
    * 
    FROM
    `tran_1`
"""

In [ ]:
cursor.execute(sql_query)

In [72]:
cursor2.execute(sql_query)

5000

In [74]:
# union 결합 
# 2개의 table을 단순하게 행으로 결합 
# pandas에서는 concat() 같은 기능
union_query = """
    SELECT 
    * 
    FROM 
    `tran_1` 
    UNION 
    SELECT 
    *
    FROM 
    `tran_2`
"""
cursor2.execute(union_query)

6786

In [ ]:
pd.DataFrame(cursor2.fetchall())

In [76]:
_db.close()
_db2.close()

In [ ]:
# Database에 접속하여 query문을 보내는 함수까지 결합하여 Class 생성

# class 키워드를 통해서 class 선언


class MyDB:
    # 독립적으로 저장이 되는 변수를 생성 
    # class가 생성되는 과정에서 바로 호출이 되는 함수 : 생성자 함수 
    # 변수 : 서버의 주소, 포트번호, 유저명, 비밀번호, 데이터베이스명
    # 변수들의 기본값을 설정 ( 로컬 피씨씨 ) 
    def __init__(self, 
                 host = "127.0.0.1", 
                 port = 3306, 
                 user = 'root', 
                 pwd = '1234', 
                 db = 'ubion'):
        self.host = host
        self.port = port
        self.user = user
        self.pwd = pwd
        self.db = db
    
    # 서버와 연결하고 커서를 생성하는 함수 
    def connect_sql(self):
        # 서버와 연결 
        self.server = pymysql.connect(
            host = self.host, 
            port = self.port, 
            user = self.user, 
            password = self.pwd, 
            db = self.db
        )
        # 커서를 생성 (독립적)
        self.cursor = self.server.cursor(pymysql.cursors.DictCursor)
    # 서버와 연결을 종료하는 함수 
    def close_sql(self):
        self.server.close()

    # 쿼리문을 실행하는 함수 
    def execute_query(self, sql_query, *values):
        # 커서에 sql_query를 보낸다. 
        self.cursor.execute(sql_query, values)
        # 커서에서 데이터를 가져온다. 
        sql_data = self.cursor.fetchall()
        # 데이터프레임으로 변환
        result = pd.DataFrame(sql_data)
        return result